In [1]:
import pandas as pd 
import json
from kafka import KafkaProducer
from time import time

In [ ]:
!wget -O  /home/sinatavakoli2022/data-engineering/homeworks/homework-streaming/green_tripdata_2019-10.csv.gz https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/green_tripdata_2019-10.csv.gz
!gunzip green_tripdata_2019-10.csv.gz


In [16]:
columns_to_keep = ['lpep_pickup_datetime','lpep_dropoff_datetime','PULocationID','DOLocationID','passenger_count','trip_distance','tip_amount']
file_path = '/home/sinatavakoli2022/data-engineering/homeworks/homework-streaming/green_tripdata_2019-10.csv'
data = pd.read_csv(file_path,usecols=columns_to_keep)
data

,lpep_pickup_datetime,lpep_dropoff_datetime,PULocationID,DOLocationID,passenger_count,trip_distance,tip_amount
0,2019-10-01 00:26:02,2019-10-01 00:39:58,112,196,1.0,5.88,0.00
1,2019-10-01 00:18:11,2019-10-01 00:22:38,43,263,1.0,0.80,0.00
2,2019-10-01 00:09:31,2019-10-01 00:24:47,255,228,2.0,7.50,0.00
3,2019-10-01 00:37:40,2019-10-01 00:41:49,181,181,1.0,0.90,0.00
4,2019-10-01 00:08:13,2019-10-01 00:17:56,97,188,1.0,2.52,2.26
...,...,...,...,...,...,...,...
476381,2019-10-31 23:30:00,2019-11-01 00:00:00,65,102,NaN,7.04,0.00
476382,2019-10-31 23:03:00,2019-10-31 23:24:00,129,136,NaN,0.00,0.00
476383,2019-10-31 23:02:00,2019-10-31 23:23:00,61,222,NaN,3.90,0.00
476384,2019-10-31 23:42:00,2019-10-31 23:56:00,76,39,NaN,3.08,0.00


In [3]:
def json_serializer(data):
    return json.dumps(data).encode('utf-8')

server = 'localhost:9092'

producer = KafkaProducer(
    bootstrap_servers=[server],
    value_serializer=json_serializer
)

producer.bootstrap_connected()

True

In [4]:
t0 = time()

for index, row in data.iterrows() : 
    message = row.to_dict()
    producer.send(topic='green-trips',value=message)
    producer.flush()
t1 = time()
took = t1 - t0
print(f'the time that it takes for transmiting the data is equal to : {took}')

the time that it takes for transmiting the data is equal to : 562.2665700912476


In [5]:
data

,lpep_pickup_datetime,lpep_dropoff_datetime,PULocationID,DOLocationID,passenger_count,trip_distance,tip_amount
0,2019-10-01 00:26:02,2019-10-01 00:39:58,112,196,1.0,5.88,0.00
1,2019-10-01 00:18:11,2019-10-01 00:22:38,43,263,1.0,0.80,0.00
2,2019-10-01 00:09:31,2019-10-01 00:24:47,255,228,2.0,7.50,0.00
3,2019-10-01 00:37:40,2019-10-01 00:41:49,181,181,1.0,0.90,0.00
4,2019-10-01 00:08:13,2019-10-01 00:17:56,97,188,1.0,2.52,2.26
...,...,...,...,...,...,...,...
476381,2019-10-31 23:30:00,2019-11-01 00:00:00,65,102,NaN,7.04,0.00
476382,2019-10-31 23:03:00,2019-10-31 23:24:00,129,136,NaN,0.00,0.00
476383,2019-10-31 23:02:00,2019-10-31 23:23:00,61,222,NaN,3.90,0.00
476384,2019-10-31 23:42:00,2019-10-31 23:56:00,76,39,NaN,3.08,0.00


In [22]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 476386 entries, 0 to 476385
Data columns (total 7 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   lpep_pickup_datetime   476386 non-null  datetime64[ns]
 1   lpep_dropoff_datetime  476386 non-null  datetime64[ns]
 2   PULocationID           476386 non-null  int64         
 3   DOLocationID           476386 non-null  int64         
 4   passenger_count        387007 non-null  float64       
 5   trip_distance          476386 non-null  float64       
 6   tip_amount             476386 non-null  float64       
dtypes: datetime64[ns](2), float64(3), int64(2)
memory usage: 25.4 MB


In [21]:
data['lpep_pickup_datetime']=pd.to_datetime(data['lpep_pickup_datetime'])
data['lpep_dropoff_datetime']=pd.to_datetime(data['lpep_dropoff_datetime'])

In [23]:
pd.io.json.build_table_schema(data)

{'fields': [{'name': 'index', 'type': 'integer'},
  {'name': 'lpep_pickup_datetime', 'type': 'datetime'},
  {'name': 'lpep_dropoff_datetime', 'type': 'datetime'},
  {'name': 'PULocationID', 'type': 'integer'},
  {'name': 'DOLocationID', 'type': 'integer'},
  {'name': 'passenger_count', 'type': 'number'},
  {'name': 'trip_distance', 'type': 'number'},
  {'name': 'tip_amount', 'type': 'number'}],
 'primaryKey': ['index'],
 'pandas_version': '1.4.0'}